In [1]:
import sys
import os
import json
import requests
import math
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

## LIQUIDEZ

### SaucerSwap

In [2]:
from eth_utils import keccak as _keccak
from brain.adapters.SaucerSwap.adapter.SaucerSwapAdapter import SaucerSwapAdapter
CONFIG_PATH = "/root/Repositorios/ild/brain/adapters/SaucerSwap/config/hedera.saucerswap.yaml"
adapter = SaucerSwapAdapter(CONFIG_PATH)
# Liquidez simple USDC/WHBAR: ±5% ticks, 50/50 por valor; mint con msg.value (patrón UI)
USDC = "0.0.456858"
FEE_BPS = 1500
RANGE_PCT = 0.05
FLEX_BPS = 0
SEND = True

# Pool USDC/WHBAR
base = adapter.config.api_base.rstrip("/")
pools = requests.get(base + "/v2/pools", headers={"x-api-key": adapter.api_key}, timeout=30).json()
whbar_id = adapter._whbar_token_id()
pool = next(p for p in pools if int(p.get("fee", 0)) == FEE_BPS and { (p.get("tokenA") or {}).get("id"), (p.get("tokenB") or {}).get("id") } == {USDC, (whbar_id or "") })
POOL_ID = pool["id"]
tA, tB = pool["tokenA"], pool["tokenB"]

dec_usdc = int((tA if tA["id"] == USDC else tB)["decimals"])
dec_wh   = int((tA if tA["id"] == (whbar_id or "") else tB)["decimals"])
px_wh    = float((tA if tA["id"] == (whbar_id or "") else tB)["priceUsd"])  # 1 WHBAR ~ 1 HBAR

# Ticks ±5% y rounding a spacing
ratio = adapter.get_pool_ratio(POOL_ID)
tick_current = int(ratio["tickCurrent"])
tick_lower = tick_current - int(abs(tick_current) * RANGE_PCT)
tick_upper = tick_current + int(abs(tick_current) * RANGE_PCT)
pool_evm = adapter._hts_to_evm(pool.get("contractId", ""))
ts = adapter._pool_tick_spacing(pool_evm)
tick_lower = adapter._round_tick_to_spacing(tick_lower, ts, "floor")
tick_upper = adapter._round_tick_to_spacing(tick_upper, ts, "ceil")
if tick_lower >= tick_upper:
    tick_upper = tick_lower + ts

# Presupuesto en USD definido por el usuario y reparto 50/50 por valor (máxima flexibilidad: mins=0)
TOTAL_USD = 10.0  # <-- ajusta tu presupuesto en USD
wh_id = whbar_id or ""
# Convertir presupuesto a cantidades deseadas (no restringimos mínimos; la pool decide el consumo)
trg0_usd = 0.5 * TOTAL_USD
trg1_usd = 0.5 * TOTAL_USD
amount0_des = int(math.floor(trg0_usd * (10**dec_usdc)))
amount1_des = int(math.floor((trg1_usd / max(1e-9, px_wh)) * (10**dec_wh)))

# Balances para clamping
bals_now = adapter.get_balances([USDC])
bal_usdc_raw = int(bals_now.get(USDC, {}).get("raw", 0))
# clamp USDC; WHBAR lo aporta msg.value en el mint
amount0_des = min(amount0_des, bal_usdc_raw)

# limitar por maxLiquidityPerTick
sel_maxL = _keccak(text="maxLiquidityPerTick()")[:4]
try:
    maxL_hex = adapter._call_rpc("eth_call", [{"to": pool_evm, "data": "0x" + sel_maxL.hex()}, "latest"]) or "0x0"
    L_max = int(maxL_hex, 16)
except Exception:
    L_max = None
# calcular L y escalar si excede
sqrtP_x96 = adapter._sqrt_ratio_x96_from_tick(tick_current)
sqrtA_x96 = adapter._sqrt_ratio_x96_from_tick(tick_lower)
sqrtB_x96 = adapter._sqrt_ratio_x96_from_tick(tick_upper)
if L_max:
    L = adapter._liquidity_for_amounts(sqrtP_x96, sqrtA_x96, sqrtB_x96, amount0_des, amount1_des)
    if L > L_max and L > 0:
        scale = max(1e-6, L_max / L)
        amount0_des = max(0, int(amount0_des * scale))
        amount1_des = max(0, int(amount1_des * scale))

# Mint
quote = adapter.liquidity_quote_by_ticks(
    tokenA=(USDC if tA["id"] == USDC else wh_id), tokenB=(wh_id if tB["id"] == wh_id else USDC), fee_bps=FEE_BPS,
    tick_lower=tick_lower, tick_upper=tick_upper,
    amount0_desired=amount0_des if tA["id"] == USDC else amount1_des,
    amount1_desired=amount1_des if tB["id"] == wh_id else amount0_des,
    slippage_bps=0,
)
# Asociar LP NFT antes del mint
try:
    if getattr(adapter, "_lp_nft_id", None):
        adapter.associate_execute(adapter._lp_nft_id)
except Exception as _aexc:
    print("associate LP NFT error:", _aexc)

prep = adapter.liquidity_prepare(quote, deadline_s=300)
# Simplificación: no gestionamos HBAR para wrap; el adapter fija msg.value (UI)

try:
    if SEND:
        res = adapter.liquidity_send(prep, wait=True)
        print(json.dumps(res, indent=2, default=str))
        # Captura de datos para la celda de remove
        try:
            last_lp_token_id = int((res.get("steps") or [])[-1].get("tokenId"))
        except Exception:
            last_lp_token_id = None
        try:
            t0_evm, t1_evm = adapter._pool_token0_token1(pool_evm) or (None, None)
        except Exception:
            t0_evm, t1_evm = (None, None)
        # Variables de sesión para la celda de remove
        try:
            txh = ((res.get("steps") or [])[-1].get("mint") or {}).get("txHash")
        except Exception:
            txh = None
        last_lp_tx_hash = txh
    else:
        print(json.dumps({"prep": prep}, indent=2, default=str))
except Exception:
    # si falla, unwrap WHBAR -> HBAR como salvaguarda
    try:
        sweep = adapter._whbar_sweep_unwrap(adapter.evm_address)
        print(json.dumps({"unwrap": sweep}, indent=2, default=str))
    except Exception as exc2:
        print(json.dumps({"unwrap_error": str(exc2)}, indent=2))
    raise


2025-10-21 05:37:28,239 INFO [SaucerSwapAdapter] SaucerSwapAdapter inicializado (network=mainnet, account_id=0.0.9637418, evm=0x3845BbD7047E65A1336a42B647487f5dc4baD93A)
2025-10-21 05:37:30,485 INFO [SaucerSwapAdapter] liq_quote: tokens HTS tA=0.0.456858 tB=0.0.1456986 fee_bps=1500
2025-10-21 05:37:30,991 INFO [SaucerSwapAdapter] liq_quote: ticks [60540,66960] amounts(desired)=(5000000,2935928756) mins=(0,0) poolId=9
2025-10-21 05:37:31,823 INFO [SaucerSwapAdapter] liq_prep: npm=0x00000000000000000000000000000000003ddbb9 fee_bps=1500 ticks=[60540,66960] amounts(desired)=(5000000,2935928756) mins=(0,0) recipient=0x3845BbD7047E65A1336a42B647487f5dc4baD93A
2025-10-21 05:37:39,675 INFO [SaucerSwapAdapter] get_mint_fee: wei=1447094230000000000 source=master_chef
2025-10-21 05:37:40,374 INFO [SaucerSwapAdapter] liq_prep: gas=1375000 value=0x14151c88ffaadc00 canSend=True notes=associate_contract: {'executed': True, 'contract_id': '0.0.4053945', 'receipts': [{'batch': ['0.0.456858'], 'error': 

{
  "steps": [
    {
      "approve": {
        "txHash": "0xfce73c5e566301b30e4fe75ec218bcd3d217f81ab8693b568bbc57cbb9c26921",
        "receipt": {
          "blockHash": "0x0296629aa33a2d19555a0dd8bd5e9addece289a5cb90197c76f29b73b1141264",
          "blockNumber": "0x5229f67",
          "from": "0x3845bbd7047e65a1336a42b647487f5dc4bad93a",
          "to": "0x000000000000000000000000000000000058a2ba",
          "cumulativeGasUsed": "0x1308e",
          "gasUsed": "0x1308e",
          "contractAddress": "0x000000000000000000000000000000000058a2ba",
          "logs": [
            {
              "address": "0x0000000000000000000000000000000000163b59",
              "blockHash": "0x0296629aa33a2d19555a0dd8bd5e9addece289a5cb90197c76f29b73b1141264",
              "blockNumber": "0x5229f67",
              "blockTimestamp": "0x68f70e18",
              "data": "0x00000000000000000000000000000000000000000000000000000000012525c1",
              "logIndex": "0x0",
              "removed": false

In [ ]:
# 3) Preparar decrease (100% de la liquidez), collect, burn y sweeps
liquidity = adapter.check_position_exists_tool(last_lp_token_id).get("details")["liquidity"]
prep_rm = adapter.liquidity_decrease_prepare(
    serial=last_lp_token_id,
    liquidity=liquidity,
    amount0_min=0,
    amount1_min=0,
    deadline_s=900,
    recipient=None,
)
print(json.dumps({
    "to": prep_rm["to"],
    "gas": prep_rm["gasEstimate"],
    "data_prefix": prep_rm["data"][:66] + "...",
    "notes": prep_rm.get("notes")
}, indent=2, default=str))

# 4) Enviar
res_rm = adapter.liquidity_decrease_send(prep_rm, wait=True)
print(json.dumps(res_rm, indent=2, default=str))

### Raydium

In [ ]:
# Raydium CLMM: Alta de liquidez SOL/USDC con reparto 50/50 (~$10) y rango ±5%

from brain.adapters.Raydium.adapter.RaydiumAdapter import RaydiumAdapter

CONFIG_PATH = "/root/Repositorios/ild/brain/adapters/Raydium/config/solana.raydium.yaml"
POOL_ID = "3ucNos4NbumPLZNWztqGHNFFgkHeRMBQAVemeeomsUxv"  # SOL/USDC
USDC_MINT = "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"
SEND = True
TOTAL_USD = 10.0
SLIPPAGE_BPS = 500

adapter = RaydiumAdapter(CONFIG_PATH)
SOL_MINT = adapter.SOL_MINT

# 1) Info de la pool y fee_bps (tradeFeeRate es ppm -> bps)

pool_info = adapter.get_pool_info(POOL_ID)
fee_bps = None
try:
    tfr = (pool_info.get("config") or {}).get("tradeFeeRate")
    if tfr is not None:
        fee_bps = int(round(float(tfr) / 100.0))
except Exception:
    fee_bps = None
if fee_bps is None:
    try:
        fr = pool_info.get("feeRate")
        if fr is not None:
            fee_bps = int(round(float(fr) * 1e4))
    except Exception:
        pass
if fee_bps is None:
    raise RuntimeError("No se pudo determinar fee_bps de la pool")

# 2) Tick actual y tickSpacing, rango ±5% en ticks y snapping

# Δtick ≈ ln(1.05)/ln(1.0001)

DELTA_TICK = int(math.log(1.05) / math.log(1.0001))

# Buscar tick actual on-chain (cuenta de la pool)

tick_current = adapter.get_pool_tick_current(POOL_ID)
if tick_current is None:
    raise RuntimeError("tickCurrent no disponible on-chain para la pool")

# tickSpacing

tick_spacing = None
try:
    tick_spacing = int((pool_info.get("config") or {}).get("tickSpacing") or pool_info.get("tickSpacing"))
except Exception:
    tick_spacing = None

lo = tick_current - DELTA_TICK
hi = tick_current + DELTA_TICK
if isinstance(tick_spacing, int) and tick_spacing > 0:
    lo = adapter._snap_tick(lo, tick_spacing, "floor")
    hi = adapter._snap_tick(hi, tick_spacing, "ceil")
if lo >= hi:
    hi = lo + (tick_spacing or 1)

# 3) Precios USD para 50/50 por valor (USDC≈1.0) con fallbacks

import requests as _rq

def fetch_prices_usd(mints):
    try:
        url = "https://api-v3.raydium.io/price/usd?mints=" + ",".join(mints)
        r = _rq.get(url, timeout=10)
        r.raise_for_status()
        data = (r.json() or {}).get("data") or {}
        return {m: float(data.get(m)) for m in mints if data.get(m) is not None}
    except Exception:
        return {}

def fetch_price_jupiter_by_id(symbol: str) -> float:
    try:
        r = _rq.get("https://price.jup.ag/v6/price", params={"ids": symbol}, timeout=10)
        r.raise_for_status()
        data = (r.json() or {}).get("data") or {}
        item = data.get(symbol) or {}
        return float(item.get("price", 0.0))
    except Exception:
        return 0.0

def fetch_price_jupiter_by_mint(mint: str) -> float:
    try:
        r = _rq.get("https://price.jup.ag/v6/price", params={"mints": mint}, timeout=10)
        r.raise_for_status()
        data = (r.json() or {}).get("data") or {}
        item = data.get(mint) or {}
        return float(item.get("price", 0.0))
    except Exception:
        return 0.0

prices = fetch_prices_usd([SOL_MINT, USDC_MINT])
px_sol = float(prices.get(SOL_MINT, 0.0))
px_usdc = float(prices.get(USDC_MINT, 1.0)) or 1.0
if px_sol <= 0:
    try:
        px_sol = float((pool_info or {}).get("price", 0.0)) or 0.0
    except Exception:
        px_sol = 0.0
if px_sol <= 0:
    px_sol = fetch_price_jupiter_by_id("SOL") or fetch_price_jupiter_by_mint(SOL_MINT)
if px_sol <= 0:
    raise RuntimeError("Precio USD de SOL no disponible")

# 4) Cantidades deseadas: 50/50 del presupuesto

half = TOTAL_USD * 0.5
dec_sol = 9
dec_usdc = 6
amountA_desired = int((half / px_sol) * (10 ** dec_sol))   # SOL
amountB_desired = int((half / px_usdc) * (10 ** dec_usdc)) # USDC

quote = adapter.liquidity_quote_by_ticks(
    mintA=SOL_MINT,
    mintB=USDC_MINT,
    fee_bps=fee_bps,
    tick_lower=lo,
    tick_upper=hi,
    amountA_desired=amountA_desired,
    amountB_desired=amountB_desired,
    slippage_bps=SLIPPAGE_BPS,
)
print("QUOTE:", json.dumps({k: quote[k] for k in ("exists","fee_bps","ticks","amounts","pool")}, indent=2, default=str))
if not quote.get("exists"):
    raise RuntimeError(f"Pool inexistente o quote inválido: {quote}")

prep = adapter.liquidity_prepare(quote)
print("PREP:", json.dumps({k: prep.get(k) for k in ("canSend","notes","meta")}, indent=2, default=str))
print("tx_count:", len(prep.get("transactions", [])))

res = None
if SEND and prep.get("canSend", True):
    res = adapter.liquidity_send(prep, wait=True)
    out = {
        "signatures": (res or {}).get("signatures"),
        "position": (res or {}).get("position"),
    }
    print("SEND:", json.dumps(out, indent=2, default=str))

    # Variables de sesión para la celda de remove
    last_ray_pool_id = POOL_ID
    last_ray_position_nft_mints = ((res or {}).get("position") or {}).get("nftMints")
    last_ray_signatures = (res or {}).get("signatures")
    print("session:", json.dumps({
        "pool": last_ray_pool_id,
        "nftMints": last_ray_position_nft_mints,
        "sigs": last_ray_signatures,
    }, indent=2, default=str))

else:
    print("No se envía: canSend=false o SEND=False")

In [ ]:
# Remover la posición
prep = adapter.liquidity_prepare_remove(
    position_nft_mint=last_ray_position_nft_mints[0],
    pool_id=POOL_ID,
    slippage_bps=0,                 # mínimos a 0 para retirar todo
)
print("PREP_REMOVE:", json.dumps({
    "canSend": prep.get("canSend"),
    "meta": prep.get("meta"),
}, indent=2, default=str))
print("tx_count:", len(prep.get("transactions", [])))

res = None
if SEND and prep.get("canSend", True):
    res = adapter.liquidity_send(prep, wait=True)
    out = {
        "signatures": (res or {}).get("signatures"),
        "receipts": (res or {}).get("receipts"),
    }
    print("SEND_REMOVE:", json.dumps(out, indent=2, default=str))


## Tools